In [1]:
dir_ = 'C:/Users/jekim/OneDrive/바탕 화면/m5-forecasting-accuracy/' # input only here

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'

In [3]:
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from collections import Counter

In [4]:
train_df = pd.read_csv(raw_data_dir+'sales_train_evaluation.csv')
prices_df = pd.read_csv(raw_data_dir+'sell_prices.csv')
calendar_df = pd.read_csv(raw_data_dir+'calendar.csv')

### Item

In [37]:
mean_by_item = train_df.drop(['id', 'cat_id', 'dept_id', 'store_id', 'state_id'], axis=1)
mean_by_item = mean_by_item.groupby([mean_by_item['item_id']]).mean()
mean_by_item = mean_by_item.reset_index()
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.5,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.5,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [38]:
a = mean_by_item.drop(['item_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,1.0,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,0.6,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.5,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3
4,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,4.1,...,1.5,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [39]:
# euclidean
tskm_10 = TimeSeriesKMeans(n_clusters=10, verbose=False, random_state=123, n_jobs=-1)
y_pred_10 = tskm_10.fit_predict(a)
print('y_pred_10: ', sorted(Counter(y_pred_10).items()))

y_pred_10:  [(0, 2108), (1, 8), (2, 3), (3, 16), (4, 655), (5, 1), (6, 190), (7, 1), (8, 35), (9, 32)]


In [40]:
tskm_eu = pd.DataFrame({'tskm_10': y_pred_10})
tskm_eu

,tskm_10
0,0
1,0
2,0
3,9
4,4
...,...
3044,0
3045,0
3046,0
3047,0


In [41]:
mean_by_item = pd.concat([mean_by_item, tskm_eu], axis = 1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_10
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,9
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0


In [42]:
div_clu0 = mean_by_item[mean_by_item['tskm_10'] == 0]
div_clu0 = div_clu0.drop(['item_id', 'tskm_10'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,1.0,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,0.6,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.1,0.5,1.8,1.9,1.5,0.2,0.7,1.0,1.0,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [43]:
tskm_5 = TimeSeriesKMeans(n_clusters=5, verbose=False, random_state=123, n_jobs=-1)
y_pred_5 = tskm_5.fit_predict(div_clu0)
print('y_pred_5: ', sorted(Counter(y_pred_5).items()))

y_pred_5:  [(0, 296), (1, 496), (2, 814), (3, 164), (4, 338)]


In [44]:
tskm_5 = pd.DataFrame({'tskm_5': y_pred_5})
div_clu0 = mean_by_item[mean_by_item['tskm_10'] == 0]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, tskm_5], axis=1)
div_clu0

,index,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_10,tskm_5
0,0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,...,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0,4
1,1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0,1
2,2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,...,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0,4
3,6,FOODS_1_008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4,0,2
4,7,FOODS_1_009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.8,1.9,1.5,0.2,0.7,1.0,1.0,1.3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0,4
2104,3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0,0
2105,3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0,2
2106,3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0,2


In [45]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['tskm_5'] == 1, 'tskm_5'] = 10
div_clu0.loc[div_clu0['tskm_5'] == 2, 'tskm_5'] = 11
div_clu0.loc[div_clu0['tskm_5'] == 3, 'tskm_5'] = 12
div_clu0.loc[div_clu0['tskm_5'] == 4, 'tskm_5'] = 13
div_clu0 = div_clu0.set_index('index')
div_clu0

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_10,tskm_5
index,,,,,,,,,,,,,,,,,,,,,
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0,13
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0,10
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0,13
6,FOODS_1_008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4,0,11
7,FOODS_1_009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.8,1.9,1.5,0.2,0.7,1.0,1.0,1.3,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0,13
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0,0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0,11


In [46]:
b = pd.concat([mean_by_item, div_clu0['tskm_5']], axis = 1)
b.loc[b['tskm_10'] == 0, 'tskm_10'] = b['tskm_5']
mean_by_item = b.drop(['tskm_5'], axis=1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_10
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,13.0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,10.0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,13.0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,9.0
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,13.0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,11.0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,11.0


In [47]:
dcol=[]
for i in list(mean_by_item):
    if 'd_' in i:
        dcol.append(i)
        
mean_by_item = mean_by_item.drop(dcol, axis=1)
mean_by_item = mean_by_item.astype({'tskm_10':'int'})
mean_by_item

,item_id,tskm_10
0,FOODS_1_001,13
1,FOODS_1_002,10
2,FOODS_1_003,13
3,FOODS_1_004,9
4,FOODS_1_005,4
...,...,...
3044,HOUSEHOLD_2_512,13
3045,HOUSEHOLD_2_513,0
3046,HOUSEHOLD_2_514,11
3047,HOUSEHOLD_2_515,11


In [48]:
train_clu = pd.merge(train_df, mean_by_item, on=['item_id'])
train_clu

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_10
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,0,0,0,0,3,3,0,1,0
1,HOBBIES_1_001_CA_2_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,...,0,2,0,2,2,0,2,0,1,0
2,HOBBIES_1_001_CA_3_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,6,0,1,0,2,1,0,1,0,0
3,HOBBIES_1_001_CA_4_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,3,1,1,1,0,1,2,2,0
4,HOBBIES_1_001_TX_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,2,1,0,2,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_827_TX_2_evaluation,FOODS_3_827,FOODS_3,FOODS,TX_2,TX,0,0,0,0,...,0,0,0,0,0,0,1,0,1,12
30486,FOODS_3_827_TX_3_evaluation,FOODS_3_827,FOODS_3,FOODS,TX_3,TX,0,0,0,0,...,0,2,2,2,1,0,4,1,2,12
30487,FOODS_3_827_WI_1_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_1,WI,0,0,0,0,...,6,8,1,2,0,1,5,5,10,12
30488,FOODS_3_827_WI_2_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_2,WI,0,0,0,0,...,3,3,0,2,0,1,3,4,4,12


In [49]:
train_clu.to_csv(raw_data_dir+'sales_train_item_tskm_eu_evaluation.csv')

### item - dtw

In [5]:
mean_by_item = train_df.drop(['id', 'cat_id', 'dept_id', 'store_id', 'state_id'], axis=1)
mean_by_item = mean_by_item.groupby([mean_by_item['item_id']]).mean()
mean_by_item = mean_by_item.reset_index()
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.5,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.5,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [6]:
a = mean_by_item.drop(['item_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,1.0,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,0.6,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.5,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3
4,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,4.1,...,1.5,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [7]:
# dtw
tskm_10 = TimeSeriesKMeans(n_clusters=10, verbose=False, random_state=123, n_jobs=-1, metric='dtw')
y_pred_10 = tskm_10.fit_predict(a)
print('y_pred_10: ', sorted(Counter(y_pred_10).items()))

MemoryError: Unable to allocate 28.7 MiB for an array with shape (1941, 1941) and data type float64

In [ ]:
tskm_dtw = pd.DataFrame({'tskm_10': y_pred_10})
mean_by_item = pd.concat([mean_by_item, tskm_dtw], axis = 1)
mean_by_item

In [ ]:
div_clu0 = mean_by_item[mean_by_item['tskm_10'] == 0]
div_clu0 = div_clu0.drop(['item_id', 'tskm_10'], axis=1)
div_clu0

### Dept & Store

In [128]:
mean_by_dept_store = train_df.drop(['id', 'item_id', 'cat_id', 'state_id'], axis=1)
mean_by_dept_store = mean_by_dept_store.groupby([mean_by_dept_store['dept_id'], mean_by_dept_store['store_id']]).mean()

In [129]:
mean_by_dept_store = mean_by_dept_store.reset_index()
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [130]:
a = mean_by_dept_store.drop(['dept_id', 'store_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,1.134259,0.814815,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,1.449074,1.259259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,1.870370,1.342593,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,0.402778,0.305556,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,0.550926,0.439815,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,0.365049,0.285437,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,0.262136,0.182524,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,0.186408,0.163107,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,0.122330,0.128155,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [114]:
# euclidean
tskm_10 = TimeSeriesKMeans(n_clusters=10, verbose=False, random_state=123, n_jobs=-1)
y_pred_10 = tskm_10.fit_predict(a)
tskm_eu = pd.DataFrame({'tskm_10': y_pred_10})
tskm_eu

In [131]:
mean_by_dept_store = pd.concat([mean_by_dept_store, tskm_eu], axis = 1)
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_5,tskm_7,tskm_10,tskm_30,tskm_50
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,1.837963,1.527778,1.435185,1.976852,1.546296,1,5,9,18,18
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,1.796296,1.870370,2.703704,3.745370,3.402778,1,1,1,24,24
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,1.597222,1.888889,2.115741,2.657407,3.013889,1,1,1,14,14
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.129630,1.027778,1.296296,1.319444,1.486111,2,2,2,21,39
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.115741,1.018519,1.384259,1.291667,1.842593,2,2,2,21,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.341748,0.324272,0.493204,0.477670,0.533981,0,0,0,0,0
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.304854,0.337864,0.392233,0.413592,0.427184,0,0,0,0,0
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.267961,0.339806,0.349515,0.485437,0.477670,0,0,0,0,0
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.304854,0.384466,0.337864,0.372816,0.380583,0,0,0,0,0


In [132]:
dcol=[]
for i in list(mean_by_dept_store):
    if 'd_' in i:
        dcol.append(i)
        
mean_by_dept_store = mean_by_dept_store.drop(dcol, axis=1)
mean_by_dept_store

,dept_id,store_id,tskm_5,tskm_7,tskm_10,tskm_30,tskm_50
0,FOODS_1,CA_1,1,5,9,18,18
1,FOODS_1,CA_2,1,1,1,24,24
2,FOODS_1,CA_3,1,1,1,14,14
3,FOODS_1,CA_4,2,2,2,21,39
4,FOODS_1,TX_1,2,2,2,21,21
...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0,0,0,0,0
66,HOUSEHOLD_2,TX_3,0,0,0,0,0
67,HOUSEHOLD_2,WI_1,0,0,0,0,0
68,HOUSEHOLD_2,WI_2,0,0,0,0,0


In [133]:
train_clu = pd.merge(train_df, mean_by_dept_store, on=['dept_id', 'store_id'])
train_clu

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_5,tskm_7,tskm_10,tskm_30,tskm_50
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,3,3,0,1,2,2,2,2,2
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,2,2,2,2,2
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,3,0,1,2,2,2,2,2
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,2,6,2,2,2,2,2
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,2,1,0,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,0,1,1,4,4,5,15,15
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,1,0,1,0,4,4,5,15,15
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,1,0,1,0,2,4,4,5,15,15
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,1,1,1,0,4,4,5,15,15


In [138]:
train_clu.to_csv(raw_data_dir+'sales_train_deptstore_tskm_eu_evaluation.csv')

### Dept & store - dtw

In [6]:
mean_by_dept_store = train_df.drop(['id', 'item_id', 'cat_id', 'state_id'], axis=1)
mean_by_dept_store = mean_by_dept_store.groupby([mean_by_dept_store['dept_id'], mean_by_dept_store['store_id']]).mean()

In [7]:
mean_by_dept_store = mean_by_dept_store.reset_index()
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [8]:
a = mean_by_dept_store.drop(['dept_id', 'store_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,1.134259,0.814815,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,1.449074,1.259259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,1.870370,1.342593,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,0.402778,0.305556,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,0.550926,0.439815,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,0.365049,0.285437,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,0.262136,0.182524,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,0.186408,0.163107,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,0.122330,0.128155,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [9]:
# dtw
tskm_10 = TimeSeriesKMeans(n_clusters=10, verbose=False, random_state=123, n_jobs=-1, metric = 'dtw')
y_pred_10 = tskm_10.fit_predict(a)
tskm_dtw = pd.DataFrame({'tskm_dtw_10': y_pred_10})
tskm_dtw

,tskm_dtw_10
0,9
1,5
2,5
3,6
4,2
...,...
65,0
66,0
67,0
68,0


In [10]:
print('y_pred_10: ', sorted(Counter(y_pred_10).items()))

y_pred_10:  [(0, 19), (1, 2), (2, 12), (3, 2), (4, 1), (5, 6), (6, 14), (7, 1), (8, 1), (9, 12)]


In [11]:
mean_by_dept_store = pd.concat([mean_by_dept_store, tskm_dtw], axis = 1)
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_dtw_10
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,9
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778,5
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889,5
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,6
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981,0
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184,0
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670,0
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583,0


In [12]:
dcol=[]
for i in list(mean_by_dept_store):
    if 'd_' in i:
        dcol.append(i)
        
mean_by_dept_store = mean_by_dept_store.drop(dcol, axis=1)
mean_by_dept_store

,dept_id,store_id,tskm_dtw_10
0,FOODS_1,CA_1,9
1,FOODS_1,CA_2,5
2,FOODS_1,CA_3,5
3,FOODS_1,CA_4,6
4,FOODS_1,TX_1,2
...,...,...,...
65,HOUSEHOLD_2,TX_2,0
66,HOUSEHOLD_2,TX_3,0
67,HOUSEHOLD_2,WI_1,0
68,HOUSEHOLD_2,WI_2,0


In [13]:
train_clu = pd.merge(train_df, mean_by_dept_store, on=['dept_id', 'store_id'])
train_clu

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_dtw_10
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,0,0,0,0,3,3,0,1,2
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,0,0,0,0,0,2
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,0,0,0,2,3,0,1,2
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,4,0,1,3,0,2,6,2
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,2,1,0,0,2,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,3,0,1,1,0,0,1,1,7
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,1,0,1,0,7
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,1,2,0,1,0,1,0,2,7
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,4,6,0,1,1,1,0,7


In [14]:
train_clu.to_csv(raw_data_dir+'sales_train_deptstore_tskm_dtw_evaluation.csv')

In [17]:
line = train_clu[train_clu['tskm_dtw_10'] == 0]
line = line.drop(['id', 'item_id','dept_id','cat_id','store_id','state_id','tskm_dtw_10'], axis=1)
line

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
416,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
417,4,0,0,1,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
418,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0
419,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,3,1
420,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29048,0,2,0,0,0,0,0,3,0,0,...,2,0,0,3,0,1,0,0,0,0
29049,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
29050,0,1,1,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,1,0,0
29051,0,0,0,0,0,0,0,0,0,0,...,0,0,1,4,1,0,0,0,0,0


### store dept - k medoids

In [5]:
mean_by_dept_store = train_df.drop(['id', 'item_id', 'cat_id', 'state_id'], axis=1)
mean_by_dept_store = mean_by_dept_store.groupby([mean_by_dept_store['dept_id'], mean_by_dept_store['store_id']]).mean()

In [6]:
mean_by_dept_store = mean_by_dept_store.reset_index()
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [7]:
a = mean_by_dept_store.drop(['dept_id', 'store_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,1.134259,0.814815,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,1.449074,1.259259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,1.870370,1.342593,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,0.402778,0.305556,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,0.550926,0.439815,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,0.365049,0.285437,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,0.262136,0.182524,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,0.186408,0.163107,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,0.122330,0.128155,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [8]:
from pyclustering.cluster.kmedoids import kmedoids
from sklearn.metrics.pairwise import euclidean_distances

In [23]:
sample = a.values.tolist()

In [24]:
initial_medoids = [0 for i in range(1941)]
kmedoids_instance = kmedoids(sample, initial_medoids)
kmedoids_instance.process()
clusters = kmedoids_instance.get_clusters()
print(clusters)

[[61], [20], [59], [63], [26], [22], [18], [28], [54], [29], [21], [57], [52], [15], [12], [2], [27], [13], [25], [5], [1], [8], [17], [30], [58], [6], [7], [11], [19], [9], [23], [3], [10], [24], [32], [51], [31], [4], [55], [37], [14], [36], [16], [50], [60], [35], [33], [56], [39], [46], [38], [62], [53], [34], [43], [45], [42], [66], [41], [44], [40], [49], [47], [48], [65], [69], [67], [68], [64], [0]]


In [16]:
origin =  [0 for i in range(1941)]
diss = euclidean_distances(a)
pd.DataFrame(diss)

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.000000,25.831926,24.178457,27.753639,26.304223,19.193118,22.910986,15.115578,18.001073,26.240160,...,48.400128,43.423121,39.401228,55.217957,52.799942,49.996710,52.979312,53.218920,55.026546,54.303560
1,25.831926,0.000000,19.353478,44.412020,42.723375,30.107900,39.386399,29.308102,29.742337,39.432926,...,64.905755,59.329362,55.482987,71.765566,69.260448,66.373827,69.531409,69.828930,71.591109,70.632508
2,24.178457,19.353478,0.000000,45.270293,42.725554,30.226204,38.883221,29.718351,28.832103,40.248601,...,66.387765,61.006063,56.908044,73.180965,70.676183,67.831967,70.927950,71.210654,73.015676,72.141391
3,27.753639,44.412020,45.270293,0.000000,10.459094,23.922312,13.979044,21.502461,25.257981,15.974026,...,23.603970,19.047813,15.401631,30.019636,27.661498,25.042078,27.840874,28.170052,29.840714,29.255885
4,26.304223,42.723375,42.725554,10.459094,0.000000,20.944065,13.282835,20.673517,23.591623,14.166009,...,26.470285,21.789508,18.343908,32.845826,30.410782,27.821947,30.689434,31.002150,32.634025,31.968521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,49.996710,66.373827,67.831967,25.042078,27.821947,43.659670,33.988449,43.010344,47.128452,32.463241,...,3.168098,8.197426,12.086382,6.106479,4.105665,0.000000,4.005676,4.601741,6.163457,5.078925
66,52.979312,69.531409,70.927950,27.840874,30.689434,46.795674,36.638912,46.056989,49.936310,35.430939,...,5.389069,11.182877,15.118895,3.136911,2.446885,4.005676,0.000000,2.583157,3.276595,2.993277
67,53.218920,69.828930,71.210654,28.170052,31.002150,47.117452,36.892292,46.174697,50.224109,35.657673,...,5.777906,11.485986,15.548787,3.469019,3.088162,4.601741,2.583157,0.000000,3.479219,3.055922
68,55.026546,71.591109,73.015676,29.840714,32.634025,48.720282,38.528562,48.083007,51.754282,37.390425,...,7.580152,13.385588,17.382848,2.581390,3.744924,6.163457,3.276595,3.479219,0.000000,3.301073


In [18]:
fp = kmedoids(diss, 100)
cluster = fp.get_clusters()
print(cluster)

TypeError: 'int' object is not subscriptable

In [23]:
kmedo_10 = kmedoids(a, [0,70])
kmedo_10.process()
y_pred_10 = kmedo_10.get_clusters()
kmedo_storedept10_eu = pd.DataFrame({'kmedo_storedept10_eu': y_pred_10})
kmedo_storedept10_eu

KeyError: 0

### store dept k-means

In [5]:
mean_by_dept_store = train_df.drop(['id', 'item_id', 'cat_id', 'state_id'], axis=1)
mean_by_dept_store = mean_by_dept_store.groupby([mean_by_dept_store['dept_id'], mean_by_dept_store['store_id']]).mean()
mean_by_dept_store = mean_by_dept_store.reset_index()
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [6]:
a = mean_by_dept_store.drop(['dept_id', 'store_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,1.134259,0.814815,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,1.449074,1.259259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,1.870370,1.342593,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,0.402778,0.305556,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,0.550926,0.439815,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,0.365049,0.285437,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,0.262136,0.182524,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,0.186408,0.163107,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,0.122330,0.128155,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [7]:
from sklearn.cluster import KMeans

In [8]:
model = KMeans(n_clusters = 10, random_state = 123)
model.fit(a)
clusters = model.fit_predict(a)
clusters

array([9, 1, 1, 2, 2, 9, 2, 9, 9, 2, 4, 7, 1, 7, 7, 4, 4, 2, 8, 4, 5, 1,
       3, 9, 1, 5, 1, 9, 6, 5, 2, 7, 9, 7, 7, 7, 7, 7, 0, 7, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 9, 9, 6, 7, 9, 9, 9, 2, 9, 2, 0, 7, 7, 0, 0, 0,
       0, 0, 0, 0])

In [10]:
km_storedept10 = pd.DataFrame({'km_storedept_10': clusters})
km_storedept10

,km_storedept_10
0,9
1,1
2,1
3,2
4,2
...,...
65,0
66,0
67,0
68,0


In [12]:
print('y_pred_10: ', sorted(Counter(clusters).items()))

y_pred_10:  [(0, 19), (1, 6), (2, 8), (3, 1), (4, 4), (5, 3), (6, 2), (7, 13), (8, 1), (9, 13)]


In [13]:
mean_by_dept_store = pd.concat([mean_by_dept_store, km_storedept10], axis = 1)
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,km_storedept_10
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,9
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778,1
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889,1
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,2
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981,0
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184,0
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670,0
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583,0


In [14]:
dcol=[]
for i in list(mean_by_dept_store):
    if 'd_' in i:
        dcol.append(i)
        
mean_by_dept_store = mean_by_dept_store.drop(dcol, axis=1)
mean_by_dept_store

,dept_id,store_id,km_storedept_10
0,FOODS_1,CA_1,9
1,FOODS_1,CA_2,1
2,FOODS_1,CA_3,1
3,FOODS_1,CA_4,2
4,FOODS_1,TX_1,2
...,...,...,...
65,HOUSEHOLD_2,TX_2,0
66,HOUSEHOLD_2,TX_3,0
67,HOUSEHOLD_2,WI_1,0
68,HOUSEHOLD_2,WI_2,0


In [15]:
train_clu = pd.merge(train_df, mean_by_dept_store, on=['dept_id', 'store_id'])
train_clu

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,km_storedept_10
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,0,0,0,0,3,3,0,1,2
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,0,0,0,0,0,2
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,0,0,0,2,3,0,1,2
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,4,0,1,3,0,2,6,2
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,2,1,0,0,2,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,3,0,1,1,0,0,1,1,5
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,1,0,1,0,5
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,1,2,0,1,0,1,0,2,5
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,4,6,0,1,1,1,0,5


In [17]:
train_clu.to_csv(raw_data_dir+'sales_train_deptstore_km_evaluation.csv')

### k-means item

In [21]:
mean_by_item = train_df.drop(['id', 'cat_id', 'dept_id', 'store_id', 'state_id'], axis=1)
mean_by_item = mean_by_item.groupby([mean_by_item['item_id']]).mean()
mean_by_item = mean_by_item.reset_index()
a = mean_by_item.drop(['item_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,1.0,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,0.6,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.5,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3
4,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,4.1,...,1.5,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [22]:
model = KMeans(n_clusters=10, random_state=123)
model.fit(a)
clusters = model.fit_predict(a)
print('y_pred_10: ', sorted(Counter(clusters).items()))

C:\Users\jekim\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\jekim\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


y_pred_10:  [(0, 190), (1, 2046), (2, 4), (3, 1), (4, 681), (5, 1), (6, 22), (7, 25), (8, 6), (9, 73)]


In [23]:
km_item_10 = pd.DataFrame({'km_item_10': clusters})
mean_by_item = pd.concat([mean_by_item, km_item_10], axis = 1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,km_item_10
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,1
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,1
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,1
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,7
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,1
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,1
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,1
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,1


In [24]:
div_clu0 = mean_by_item[mean_by_item['km_item_10'] == 1]
div_clu0 = div_clu0.drop(['item_id', 'km_item_10'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,1.0,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,0.6,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.1,0.5,1.8,1.9,1.5,0.2,0.7,1.0,1.0,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [25]:
km_5 = KMeans(n_clusters=5, random_state=123)
y_pred_5 = km_5.fit_predict(div_clu0)
print('y_pred_5: ', sorted(Counter(y_pred_5).items()))

C:\Users\jekim\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


y_pred_5:  [(0, 166), (1, 461), (2, 208), (3, 319), (4, 892)]


In [26]:
km_5 = pd.DataFrame({'km_5': y_pred_5})
div_clu0 = mean_by_item[mean_by_item['km_item_10'] == 1]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, km_5], axis=1)
div_clu0

,index,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,km_item_10,km_5
0,0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,...,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,1,3
1,1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,1,1
2,2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,...,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,1,3
3,6,FOODS_1_008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4,1,4
4,7,FOODS_1_009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.8,1.9,1.5,0.2,0.7,1.0,1.0,1.3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,1,3
2042,3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,1,2
2043,3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,1,4
2044,3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,1,4


In [27]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['km_5'] == 0, 'km_5'] = 10
div_clu0.loc[div_clu0['km_5'] == 2, 'km_5'] = 11
div_clu0.loc[div_clu0['km_5'] == 3, 'km_5'] = 12
div_clu0.loc[div_clu0['km_5'] == 4, 'km_5'] = 13
div_clu0 = div_clu0.set_index('index')
div_clu0

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,km_item_10,km_5
index,,,,,,,,,,,,,,,,,,,,,
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,1,12
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,1,1
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,1,12
6,FOODS_1_008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4,1,13
7,FOODS_1_009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.8,1.9,1.5,0.2,0.7,1.0,1.0,1.3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,1,12
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,1,11
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,1,13


In [28]:
b = pd.concat([mean_by_item, div_clu0['km_5']], axis = 1)
b.loc[b['km_item_10'] == 1, 'km_item_10'] = b['km_5']
mean_by_item = b.drop(['km_5'], axis=1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,km_item_10
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,12.0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,1.0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,12.0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,7.0
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,12.0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,11.0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,13.0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,13.0


In [29]:
dcol=[]
for i in list(mean_by_item):
    if 'd_' in i:
        dcol.append(i)
        
mean_by_item = mean_by_item.drop(dcol, axis=1)
mean_by_item = mean_by_item.astype({'km_item_10':'int'})
mean_by_item

,item_id,km_item_10
0,FOODS_1_001,12
1,FOODS_1_002,1
2,FOODS_1_003,12
3,FOODS_1_004,7
4,FOODS_1_005,4
...,...,...
3044,HOUSEHOLD_2_512,12
3045,HOUSEHOLD_2_513,11
3046,HOUSEHOLD_2_514,13
3047,HOUSEHOLD_2_515,13


In [30]:
train_clu = pd.merge(train_df, mean_by_item, on=['item_id'])
train_clu

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,km_item_10
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,0,0,0,0,3,3,0,1,13
1,HOBBIES_1_001_CA_2_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,...,0,2,0,2,2,0,2,0,1,13
2,HOBBIES_1_001_CA_3_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,6,0,1,0,2,1,0,1,0,13
3,HOBBIES_1_001_CA_4_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,3,1,1,1,0,1,2,2,13
4,HOBBIES_1_001_TX_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,2,1,0,2,1,0,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_827_TX_2_evaluation,FOODS_3_827,FOODS_3,FOODS,TX_2,TX,0,0,0,0,...,0,0,0,0,0,0,1,0,1,10
30486,FOODS_3_827_TX_3_evaluation,FOODS_3_827,FOODS_3,FOODS,TX_3,TX,0,0,0,0,...,0,2,2,2,1,0,4,1,2,10
30487,FOODS_3_827_WI_1_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_1,WI,0,0,0,0,...,6,8,1,2,0,1,5,5,10,10
30488,FOODS_3_827_WI_2_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_2,WI,0,0,0,0,...,3,3,0,2,0,1,3,4,4,10


In [31]:
train_clu.to_csv(raw_data_dir+'sales_train_item_km_evaluation.csv')

### DBSCAN - item

In [62]:
mean_by_item = train_df.drop(['id', 'cat_id', 'dept_id', 'store_id', 'state_id'], axis=1)
mean_by_item = mean_by_item.groupby([mean_by_item['item_id']]).mean()
mean_by_item = mean_by_item.reset_index()
a = mean_by_item.drop(['item_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,1.0,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,0.6,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.5,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3
4,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,4.1,...,1.5,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [84]:
model = DBSCAN(eps=100,min_samples=3)
model.fit(a)
clusters = model.fit_predict(a)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 154), (0, 2892), (1, 3)]


In [85]:
dbscan_item = pd.DataFrame({'dbscan_item': clusters})
mean_by_item = pd.concat([mean_by_item, dbscan_item], axis = 1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,-1
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0


In [86]:
div_clu0 = mean_by_item[mean_by_item['dbscan_item'] == -1]
div_clu0 = div_clu0.drop(['item_id', 'dbscan_item'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.5,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3
16,4.7,2.6,3.0,2.4,1.3,1.8,3.0,3.2,3.1,1.6,...,10.4,13.4,14.6,9.8,9.2,7.1,9.9,10.0,12.8,10.5
17,10.6,10.4,4.7,6.9,5.1,5.6,5.7,10.0,7.4,8.2,...,8.3,7.9,10.9,4.5,4.7,3.9,6.2,7.3,9.2,5.4
30,6.2,6.3,3.4,5.4,4.8,4.5,4.9,4.2,3.1,2.5,...,10.0,9.8,7.1,7.1,6.2,5.2,7.5,7.2,8.9,8.3
41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.6,10.9,13.9,10.4,6.8,5.4,10.1,11.5,10.4,12.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.0,13.9,15.7,10.6,6.1,6.2,7.8,10.8,9.9,9.1
2466,7.6,5.4,3.7,2.6,4.8,4.3,2.6,6.4,4.8,12.7,...,3.1,3.5,7.2,3.5,2.6,1.4,2.0,2.7,5.1,5.3
2506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.8,6.6,8.1,5.1,4.2,3.6,3.1,5.1,5.0,7.8
2513,5.3,4.8,3.7,3.6,4.3,4.8,4.2,5.9,4.7,4.2,...,6.3,8.3,7.0,4.4,5.2,4.6,3.4,7.4,6.7,4.7


In [90]:
model = DBSCAN(eps=200,min_samples=3)
model.fit(div_clu0)
clusters = model.fit_predict(div_clu0)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 44), (0, 107), (1, 3)]


In [91]:
dbscan_2 = pd.DataFrame({'dbscan2': clusters})
div_clu0 = mean_by_item[mean_by_item['dbscan_item'] == -1]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, dbscan_2], axis=1)
div_clu0

,index,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item,dbscan2
0,3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,-1,-1
1,16,FOODS_1_018,4.7,2.6,3.0,2.4,1.3,1.8,3.0,3.2,...,14.6,9.8,9.2,7.1,9.9,10.0,12.8,10.5,-1,0
2,17,FOODS_1_019,10.6,10.4,4.7,6.9,5.1,5.6,5.7,10.0,...,10.9,4.5,4.7,3.9,6.2,7.3,9.2,5.4,-1,0
3,30,FOODS_1_032,6.2,6.3,3.4,5.4,4.8,4.5,4.9,4.2,...,7.1,7.1,6.2,5.2,7.5,7.2,8.9,8.3,-1,0
4,41,FOODS_1_043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.9,10.4,6.8,5.4,10.1,11.5,10.4,12.8,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2457,HOUSEHOLD_1_465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.7,10.6,6.1,6.2,7.8,10.8,9.9,9.1,-1,0
150,2466,HOUSEHOLD_1_474,7.6,5.4,3.7,2.6,4.8,4.3,2.6,6.4,...,7.2,3.5,2.6,1.4,2.0,2.7,5.1,5.3,-1,0
151,2506,HOUSEHOLD_1_514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.1,5.1,4.2,3.6,3.1,5.1,5.0,7.8,-1,0
152,2513,HOUSEHOLD_1_521,5.3,4.8,3.7,3.6,4.3,4.8,4.2,5.9,...,7.0,4.4,5.2,4.6,3.4,7.4,6.7,4.7,-1,0


In [92]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['dbscan2'] == 0, 'dbscan2'] = 2
div_clu0.loc[div_clu0['dbscan2'] == 1, 'dbscan2'] = 3
div_clu0 = div_clu0.set_index('index')
div_clu0

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item,dbscan2
index,,,,,,,,,,,,,,,,,,,,,
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,-1,-1
16,FOODS_1_018,4.7,2.6,3.0,2.4,1.3,1.8,3.0,3.2,3.1,...,14.6,9.8,9.2,7.1,9.9,10.0,12.8,10.5,-1,2
17,FOODS_1_019,10.6,10.4,4.7,6.9,5.1,5.6,5.7,10.0,7.4,...,10.9,4.5,4.7,3.9,6.2,7.3,9.2,5.4,-1,2
30,FOODS_1_032,6.2,6.3,3.4,5.4,4.8,4.5,4.9,4.2,3.1,...,7.1,7.1,6.2,5.2,7.5,7.2,8.9,8.3,-1,2
41,FOODS_1_043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.9,10.4,6.8,5.4,10.1,11.5,10.4,12.8,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,HOUSEHOLD_1_465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.7,10.6,6.1,6.2,7.8,10.8,9.9,9.1,-1,2
2466,HOUSEHOLD_1_474,7.6,5.4,3.7,2.6,4.8,4.3,2.6,6.4,4.8,...,7.2,3.5,2.6,1.4,2.0,2.7,5.1,5.3,-1,2
2506,HOUSEHOLD_1_514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.1,5.1,4.2,3.6,3.1,5.1,5.0,7.8,-1,2


In [93]:
b = pd.concat([mean_by_item, div_clu0['dbscan2']], axis = 1)
b.loc[b['dbscan_item'] == -1, 'dbscan_item'] = b['dbscan2']
mean_by_item = b.drop(['dbscan2'], axis=1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0.0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0.0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0.0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,-1.0
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0.0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0.0


In [94]:
# 한번 더
div_clu0 = mean_by_item[mean_by_item['dbscan_item'] == -1]
div_clu0 = div_clu0.drop(['item_id', 'dbscan_item'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.5,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3
41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.6,10.9,13.9,10.4,6.8,5.4,10.1,11.5,10.4,12.8
214,5.6,4.5,4.8,2.8,2.2,4.7,4.3,5.3,3.5,3.3,...,13.2,19.3,11.2,10.3,7.3,7.3,9.0,11.4,11.8,13.9
574,21.6,25.0,15.4,23.5,14.4,24.2,19.8,23.2,12.7,16.6,...,15.1,23.3,28.1,18.1,10.2,10.3,11.3,13.5,13.2,13.3
620,13.9,16.2,15.5,13.1,12.6,20.6,13.7,10.0,13.3,11.4,...,5.0,11.3,11.5,13.4,11.6,7.0,4.6,6.0,7.9,12.1
643,13.3,8.5,5.4,16.7,15.2,18.1,22.9,27.9,14.5,19.3,...,10.4,10.6,10.6,8.2,9.8,9.2,8.8,12.2,11.4,11.5
681,2.1,0.5,0.6,0.6,0.2,0.7,0.7,1.2,0.7,0.2,...,1.4,0.9,0.8,0.2,0.2,1.8,0.6,0.7,1.2,0.2
692,21.0,14.5,13.7,10.0,14.3,16.9,16.6,23.0,17.7,11.3,...,12.1,13.7,12.4,10.5,10.3,12.1,10.1,11.6,13.9,13.9
702,104.6,103.6,67.3,64.2,53.1,87.7,111.7,131.1,130.6,51.7,...,63.8,73.6,67.5,37.4,36.9,38.0,40.0,62.6,68.2,67.9
732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.0,29.7,27.8,20.8,28.1,18.3,20.7,17.8,27.6,35.6


In [100]:
model = DBSCAN(eps=300,min_samples=3)
model.fit(div_clu0)
clusters = model.fit_predict(div_clu0)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 25), (0, 8), (1, 11)]


In [101]:
dbscan_2 = pd.DataFrame({'dbscan2': clusters})
div_clu0 = mean_by_item[mean_by_item['dbscan_item'] == -1]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, dbscan_2], axis=1)
div_clu0

,index,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item,dbscan2
0,3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,-1.0,0
1,41,FOODS_1_043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.9,10.4,6.8,5.4,10.1,11.5,10.4,12.8,-1.0,0
2,214,FOODS_1_218,5.6,4.5,4.8,2.8,2.2,4.7,4.3,5.3,...,11.2,10.3,7.3,7.3,9.0,11.4,11.8,13.9,-1.0,1
3,574,FOODS_2_360,21.6,25.0,15.4,23.5,14.4,24.2,19.8,23.2,...,28.1,18.1,10.2,10.3,11.3,13.5,13.2,13.3,-1.0,-1
4,620,FOODS_3_007,13.9,16.2,15.5,13.1,12.6,20.6,13.7,10.0,...,11.5,13.4,11.6,7.0,4.6,6.0,7.9,12.1,-1.0,1
5,643,FOODS_3_030,13.3,8.5,5.4,16.7,15.2,18.1,22.9,27.9,...,10.6,8.2,9.8,9.2,8.8,12.2,11.4,11.5,-1.0,-1
6,681,FOODS_3_069,2.1,0.5,0.6,0.6,0.2,0.7,0.7,1.2,...,0.8,0.2,0.2,1.8,0.6,0.7,1.2,0.2,-1.0,-1
7,692,FOODS_3_080,21.0,14.5,13.7,10.0,14.3,16.9,16.6,23.0,...,12.4,10.5,10.3,12.1,10.1,11.6,13.9,13.9,-1.0,1
8,702,FOODS_3_090,104.6,103.6,67.3,64.2,53.1,87.7,111.7,131.1,...,67.5,37.4,36.9,38.0,40.0,62.6,68.2,67.9,-1.0,-1
9,732,FOODS_3_120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.8,20.8,28.1,18.3,20.7,17.8,27.6,35.6,-1.0,-1


In [102]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['dbscan2'] == 0, 'dbscan2'] = 4
div_clu0.loc[div_clu0['dbscan2'] == 1, 'dbscan2'] = 5
div_clu0 = div_clu0.set_index('index')
div_clu0

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item,dbscan2
index,,,,,,,,,,,,,,,,,,,,,
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,-1.0,4
41,FOODS_1_043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.9,10.4,6.8,5.4,10.1,11.5,10.4,12.8,-1.0,4
214,FOODS_1_218,5.6,4.5,4.8,2.8,2.2,4.7,4.3,5.3,3.5,...,11.2,10.3,7.3,7.3,9.0,11.4,11.8,13.9,-1.0,5
574,FOODS_2_360,21.6,25.0,15.4,23.5,14.4,24.2,19.8,23.2,12.7,...,28.1,18.1,10.2,10.3,11.3,13.5,13.2,13.3,-1.0,-1
620,FOODS_3_007,13.9,16.2,15.5,13.1,12.6,20.6,13.7,10.0,13.3,...,11.5,13.4,11.6,7.0,4.6,6.0,7.9,12.1,-1.0,5
643,FOODS_3_030,13.3,8.5,5.4,16.7,15.2,18.1,22.9,27.9,14.5,...,10.6,8.2,9.8,9.2,8.8,12.2,11.4,11.5,-1.0,-1
681,FOODS_3_069,2.1,0.5,0.6,0.6,0.2,0.7,0.7,1.2,0.7,...,0.8,0.2,0.2,1.8,0.6,0.7,1.2,0.2,-1.0,-1
692,FOODS_3_080,21.0,14.5,13.7,10.0,14.3,16.9,16.6,23.0,17.7,...,12.4,10.5,10.3,12.1,10.1,11.6,13.9,13.9,-1.0,5
702,FOODS_3_090,104.6,103.6,67.3,64.2,53.1,87.7,111.7,131.1,130.6,...,67.5,37.4,36.9,38.0,40.0,62.6,68.2,67.9,-1.0,-1


In [103]:
b = pd.concat([mean_by_item, div_clu0['dbscan2']], axis = 1)
b.loc[b['dbscan_item'] == -1, 'dbscan_item'] = b['dbscan2']
mean_by_item = b.drop(['dbscan2'], axis=1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0.0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0.0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0.0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,4.0
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0.0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0.0


In [104]:
# label 0 재클러스터링
div_clu0 = mean_by_item[mean_by_item['dbscan_item'] == 0]
div_clu0 = div_clu0.drop(['item_id', 'dbscan_item'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,1.0,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,0.6,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
4,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,4.1,...,1.5,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9
5,1.6,0.9,2.0,1.6,1.4,2.1,5.4,2.1,2.7,1.0,...,1.5,0.8,1.6,1.7,1.2,1.3,0.9,0.9,1.1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [113]:
model = DBSCAN(eps=50,min_samples=3)
model.fit(div_clu0)
clusters = model.fit_predict(div_clu0)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 249), (0, 2639), (1, 4)]


In [114]:
dbscan_2 = pd.DataFrame({'dbscan2': clusters})
div_clu0 = mean_by_item[mean_by_item['dbscan_item'] == 0]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, dbscan_2], axis=1)
div_clu0

,index,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item,dbscan2
0,0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,...,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0.0,0
1,1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0.0,0
2,2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,...,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0.0,0
3,4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,...,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,0.0,0
4,5,FOODS_1_006,1.6,0.9,2.0,1.6,1.4,2.1,5.4,2.1,...,1.6,1.7,1.2,1.3,0.9,0.9,1.1,0.5,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2887,3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0.0,0
2888,3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0,0
2889,3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0,0
2890,3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0.0,0


In [115]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['dbscan2'] == -1, 'dbscan2'] = 6
div_clu0.loc[div_clu0['dbscan2'] == 1, 'dbscan2'] = 7
div_clu0 = div_clu0.set_index('index')
div_clu0

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item,dbscan2
index,,,,,,,,,,,,,,,,,,,,,
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0.0,0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0.0,0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0.0,0
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,0.0,0
5,FOODS_1_006,1.6,0.9,2.0,1.6,1.4,2.1,5.4,2.1,2.7,...,1.6,1.7,1.2,1.3,0.9,0.9,1.1,0.5,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0.0,0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0,0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0,0


In [116]:
b = pd.concat([mean_by_item, div_clu0['dbscan2']], axis = 1)
b.loc[b['dbscan_item'] == 0, 'dbscan_item'] = b['dbscan2']
mean_by_item = b.drop(['dbscan2'], axis=1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0.0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0.0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0.0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,4.0
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0.0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0.0


In [117]:
div_clu0 = mean_by_item[mean_by_item['dbscan_item'] == 0]
div_clu0 = div_clu0.drop(['item_id', 'dbscan_item'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,1.0,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,0.6,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
4,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,4.1,...,1.5,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9
5,1.6,0.9,2.0,1.6,1.4,2.1,5.4,2.1,2.7,1.0,...,1.5,0.8,1.6,1.7,1.2,1.3,0.9,0.9,1.1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [126]:
model = DBSCAN(eps=20,min_samples=3)
model.fit(div_clu0)
clusters = model.fit_predict(div_clu0)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 982), (0, 1651), (1, 3), (2, 3)]


In [127]:
dbscan_2 = pd.DataFrame({'dbscan2': clusters})
div_clu0 = mean_by_item[mean_by_item['dbscan_item'] == 0]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, dbscan_2], axis=1)
div_clu0

,index,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item,dbscan2
0,0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,...,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0.0,-1
1,1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0.0,0
2,2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,...,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0.0,-1
3,4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,...,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,0.0,-1
4,5,FOODS_1_006,1.6,0.9,2.0,1.6,1.4,2.1,5.4,2.1,...,1.6,1.7,1.2,1.3,0.9,0.9,1.1,0.5,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0.0,0
2635,3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0,0
2636,3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0,0
2637,3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0.0,0


In [128]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['dbscan2'] == -1, 'dbscan2'] = 8
div_clu0.loc[div_clu0['dbscan2'] == 1, 'dbscan2'] = 9
div_clu0.loc[div_clu0['dbscan2'] == 2, 'dbscan2'] = 10
div_clu0 = div_clu0.set_index('index')
div_clu0

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item,dbscan2
index,,,,,,,,,,,,,,,,,,,,,
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0.0,8
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0.0,0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0.0,8
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,0.0,8
5,FOODS_1_006,1.6,0.9,2.0,1.6,1.4,2.1,5.4,2.1,2.7,...,1.6,1.7,1.2,1.3,0.9,0.9,1.1,0.5,0.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0.0,0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0,0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0,0


In [129]:
b = pd.concat([mean_by_item, div_clu0['dbscan2']], axis = 1)
b.loc[b['dbscan_item'] == 0, 'dbscan_item'] = b['dbscan2']
mean_by_item = b.drop(['dbscan2'], axis=1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,8.0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0.0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,8.0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,4.0
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0.0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0.0


In [130]:
div_clu0 = mean_by_item[mean_by_item['dbscan_item'] == 0]
div_clu0 = div_clu0.drop(['item_id', 'dbscan_item'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.9,0.5,0.3,0.7,0.1,0.2,0.4,0.9,0.6,1.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.7,0.6,1.4,0.8,0.9,1.3,1.6,1.2,1.0,1.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.1,0.0,0.5,0.1,0.2,0.6,0.6,0.3,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [134]:
model = DBSCAN(eps=12,min_samples=3)
model.fit(div_clu0)
clusters = model.fit_predict(div_clu0)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 806), (0, 845)]


In [135]:
dbscan_2 = pd.DataFrame({'dbscan2': clusters})
div_clu0 = mean_by_item[mean_by_item['dbscan_item'] == 0]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, dbscan_2], axis=1)
div_clu0

,index,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item,dbscan2
0,1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0.0,-1
1,6,FOODS_1_008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4,0.0,0
2,8,FOODS_1_010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.7,0.1,0.2,0.4,0.9,0.6,1.0,0.0,-1
3,15,FOODS_1_017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.4,0.8,0.9,1.3,1.6,1.2,1.0,1.0,0.0,-1
4,20,FOODS_1_022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.1,0.2,0.6,0.6,0.3,0.1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0.0,-1
1647,3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0,-1
1648,3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0,0
1649,3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0.0,0


In [136]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['dbscan2'] == -1, 'dbscan2'] = 11
div_clu0 = div_clu0.set_index('index')
div_clu0

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item,dbscan2
index,,,,,,,,,,,,,,,,,,,,,
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0.0,11
6,FOODS_1_008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4,0.0,0
8,FOODS_1_010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.7,0.1,0.2,0.4,0.9,0.6,1.0,0.0,11
15,FOODS_1_017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.4,0.8,0.9,1.3,1.6,1.2,1.0,1.0,0.0,11
20,FOODS_1_022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.1,0.2,0.6,0.6,0.3,0.1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0.0,11
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0,11
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0,0


In [137]:
b = pd.concat([mean_by_item, div_clu0['dbscan2']], axis = 1)
b.loc[b['dbscan_item'] == 0, 'dbscan_item'] = b['dbscan2']
mean_by_item = b.drop(['dbscan2'], axis=1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,8.0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,11.0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,8.0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,4.0
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,11.0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,11.0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0.0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0.0


In [139]:
dcol=[]
for i in list(mean_by_item):
    if 'd_' in i:
        dcol.append(i)
        
mean_by_item = mean_by_item.drop(dcol, axis=1)
mean_by_item = mean_by_item.astype({'dbscan_item':'int'})
mean_by_item

,item_id,dbscan_item
0,FOODS_1_001,8
1,FOODS_1_002,11
2,FOODS_1_003,8
3,FOODS_1_004,4
4,FOODS_1_005,8
...,...,...
3044,HOUSEHOLD_2_512,11
3045,HOUSEHOLD_2_513,11
3046,HOUSEHOLD_2_514,0
3047,HOUSEHOLD_2_515,0


In [140]:
# item cluster 12개
train_clu = pd.merge(train_df, mean_by_item, on=['item_id'])
train_clu

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_item
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,0,0,0,0,3,3,0,1,0
1,HOBBIES_1_001_CA_2_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,...,0,2,0,2,2,0,2,0,1,0
2,HOBBIES_1_001_CA_3_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,6,0,1,0,2,1,0,1,0,0
3,HOBBIES_1_001_CA_4_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,3,1,1,1,0,1,2,2,0
4,HOBBIES_1_001_TX_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,2,1,0,2,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_827_TX_2_evaluation,FOODS_3_827,FOODS_3,FOODS,TX_2,TX,0,0,0,0,...,0,0,0,0,0,0,1,0,1,8
30486,FOODS_3_827_TX_3_evaluation,FOODS_3_827,FOODS_3,FOODS,TX_3,TX,0,0,0,0,...,0,2,2,2,1,0,4,1,2,8
30487,FOODS_3_827_WI_1_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_1,WI,0,0,0,0,...,6,8,1,2,0,1,5,5,10,8
30488,FOODS_3_827_WI_2_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_2,WI,0,0,0,0,...,3,3,0,2,0,1,3,4,4,8


In [141]:
train_clu.to_csv(raw_data_dir+'sales_train_item_dbscan_evaluation.csv')

### dbscan - store dept

In [96]:
mean_by_dept_store = train_df.drop(['id', 'item_id', 'cat_id', 'state_id'], axis=1)
mean_by_dept_store = mean_by_dept_store.groupby([mean_by_dept_store['dept_id'], mean_by_dept_store['store_id']]).mean()
mean_by_dept_store = mean_by_dept_store.reset_index()
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [97]:
a = mean_by_dept_store.drop(['dept_id', 'store_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,1.134259,0.814815,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,1.449074,1.259259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,1.870370,1.342593,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,0.402778,0.305556,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,0.550926,0.439815,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,0.365049,0.285437,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,0.262136,0.182524,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,0.186408,0.163107,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,0.122330,0.128155,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [98]:
model = DBSCAN(eps=25,min_samples=3)
model.fit(a)
clusters = model.fit_predict(a)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 4), (0, 66)]


In [99]:
dbscan_dept_store = pd.DataFrame({'dbscan_dept_store': clusters})
mean_by_dept_store = pd.concat([mean_by_dept_store, dbscan_dept_store], axis = 1)
mean_by_dept_store.loc[mean_by_dept_store['dbscan_dept_store'] == -1, 'dbscan_dept_store'] = 1
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,0
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778,0
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889,0
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,0
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981,0
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184,0
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670,0
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583,0


In [101]:
div_clu0 = mean_by_dept_store[mean_by_dept_store['dbscan_dept_store'] == 0]
div_clu0 = div_clu0.drop(['dept_id', 'store_id', 'dbscan_dept_store'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,1.134259,0.814815,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,1.449074,1.259259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,1.870370,1.342593,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,0.402778,0.305556,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,0.550926,0.439815,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,0.365049,0.285437,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,0.262136,0.182524,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,0.186408,0.163107,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,0.122330,0.128155,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [102]:
model = DBSCAN(eps=20,min_samples=3)
model.fit(div_clu0)
clusters = model.fit_predict(div_clu0)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 7), (0, 56), (1, 3)]


In [105]:
dbscan_2 = pd.DataFrame({'dbscan2': clusters})
div_clu0 = mean_by_dept_store[mean_by_dept_store['dbscan_dept_store'] == 0]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, dbscan_2], axis=1)
div_clu0

,index,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store,dbscan2
0,0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,...,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,0,0
1,1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,...,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778,0,1
2,2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,...,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889,0,1
3,3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,...,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,0,0
4,4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,...,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,...,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981,0,0
62,66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,...,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184,0,0
63,67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,...,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670,0,0
64,68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,...,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583,0,0


In [106]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['dbscan2'] == -1, 'dbscan2'] = 2
div_clu0.loc[div_clu0['dbscan2'] == 1, 'dbscan2'] = 3
div_clu0 = div_clu0.set_index('index')
div_clu0

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store,dbscan2
index,,,,,,,,,,,,,,,,,,,,,
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,0,0
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778,0,3
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889,0,3
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,0,0
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981,0,0
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184,0,0
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670,0,0


In [107]:
b = pd.concat([mean_by_dept_store, div_clu0['dbscan2']], axis = 1)
b.loc[b['dbscan_dept_store'] == 0, 'dbscan_dept_store'] = b['dbscan2']
mean_by_dept_store = b.drop(['dbscan2'], axis=1)
mean_by_dept_store
# cluster = [0, 1, 2, 3]
# cluster 0 = 56
# cluster 1 = 4
# cluster 2 = 7
# cluster 3 = 3

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,0.0
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778,3.0
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889,3.0
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,0.0
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981,0.0
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184,0.0
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670,0.0
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583,0.0


In [109]:
div_clu0 = mean_by_dept_store[mean_by_dept_store['dbscan_dept_store'] == 0]
div_clu0 = div_clu0.drop(['dept_id', 'store_id', 'dbscan_dept_store'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,1.134259,0.814815,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
3,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,0.402778,0.305556,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,0.550926,0.439815,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
5,0.972222,0.782407,0.736111,0.495370,0.592593,0.856481,0.782407,0.893519,0.833333,0.814815,...,1.615741,1.467593,1.699074,1.106481,1.222222,0.990741,1.222222,1.384259,1.800926,1.250000
6,0.638889,0.569444,0.523148,0.388889,0.282407,0.495370,0.444444,0.680556,0.615741,0.449074,...,2.009259,1.509259,1.583333,1.500000,1.379630,1.361111,1.643519,1.851852,1.444444,1.361111
7,1.587963,1.416667,0.962963,0.671296,0.000000,1.375000,1.541667,1.680556,0.731481,0.768519,...,1.736111,2.212963,1.944444,1.421296,1.578704,1.351852,1.731481,1.796296,1.750000,1.629630
8,0.884259,0.851852,0.712963,0.949074,0.444444,0.875000,0.851852,0.958333,0.606481,0.703704,...,2.157407,2.675926,2.657407,2.101852,1.787037,1.824074,1.555556,2.004630,1.976852,1.842593
9,0.703704,0.638889,0.587963,0.453704,0.402778,0.787037,0.629630,0.754630,0.717593,0.444444,...,1.555556,1.958333,1.787037,1.254630,1.291667,1.222222,1.564815,1.319444,1.458333,1.287037
10,1.693467,1.645729,0.994975,1.195980,0.889447,1.221106,1.391960,1.876884,1.459799,1.404523,...,1.369347,1.535176,1.962312,1.394472,1.195980,1.072864,1.032663,1.090452,1.575377,1.806533
11,0.532663,0.570352,0.346734,0.437186,0.256281,0.258794,0.351759,0.474874,0.437186,0.399497,...,1.281407,1.560302,2.213568,1.316583,1.155779,1.005025,1.258794,1.195980,1.886935,2.097990


In [110]:
model = DBSCAN(eps=15,min_samples=3)
model.fit(div_clu0)
clusters = model.fit_predict(div_clu0)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 5), (0, 51)]


In [111]:
dbscan_2 = pd.DataFrame({'dbscan2': clusters})
div_clu0 = mean_by_dept_store[mean_by_dept_store['dbscan_dept_store'] == 0]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, dbscan_2], axis=1)
div_clu0

,index,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store,dbscan2
0,0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,...,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,0.0,-1
1,3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,...,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,0.0,0
2,4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,...,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,0.0,0
3,5,FOODS_1,TX_2,0.972222,0.782407,0.736111,0.495370,0.592593,0.856481,0.782407,...,1.699074,1.106481,1.222222,0.990741,1.222222,1.384259,1.800926,1.250000,0.0,-1
4,6,FOODS_1,TX_3,0.638889,0.569444,0.523148,0.388889,0.282407,0.495370,0.444444,...,1.583333,1.500000,1.379630,1.361111,1.643519,1.851852,1.444444,1.361111,0.0,0
5,7,FOODS_1,WI_1,1.587963,1.416667,0.962963,0.671296,0.000000,1.375000,1.541667,...,1.944444,1.421296,1.578704,1.351852,1.731481,1.796296,1.750000,1.629630,0.0,-1
6,8,FOODS_1,WI_2,0.884259,0.851852,0.712963,0.949074,0.444444,0.875000,0.851852,...,2.657407,2.101852,1.787037,1.824074,1.555556,2.004630,1.976852,1.842593,0.0,-1
7,9,FOODS_1,WI_3,0.703704,0.638889,0.587963,0.453704,0.402778,0.787037,0.629630,...,1.787037,1.254630,1.291667,1.222222,1.564815,1.319444,1.458333,1.287037,0.0,0
8,10,FOODS_2,CA_1,1.693467,1.645729,0.994975,1.195980,0.889447,1.221106,1.391960,...,1.962312,1.394472,1.195980,1.072864,1.032663,1.090452,1.575377,1.806533,0.0,0
9,11,FOODS_2,CA_2,0.532663,0.570352,0.346734,0.437186,0.256281,0.258794,0.351759,...,2.213568,1.316583,1.155779,1.005025,1.258794,1.195980,1.886935,2.097990,0.0,0


In [112]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['dbscan2'] == -1, 'dbscan2'] = 4
div_clu0 = div_clu0.set_index('index')
div_clu0

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store,dbscan2
index,,,,,,,,,,,,,,,,,,,,,
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,0.0,4
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,0.0,0
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,0.0,0
5,FOODS_1,TX_2,0.972222,0.782407,0.736111,0.495370,0.592593,0.856481,0.782407,0.893519,...,1.699074,1.106481,1.222222,0.990741,1.222222,1.384259,1.800926,1.250000,0.0,4
6,FOODS_1,TX_3,0.638889,0.569444,0.523148,0.388889,0.282407,0.495370,0.444444,0.680556,...,1.583333,1.500000,1.379630,1.361111,1.643519,1.851852,1.444444,1.361111,0.0,0
7,FOODS_1,WI_1,1.587963,1.416667,0.962963,0.671296,0.000000,1.375000,1.541667,1.680556,...,1.944444,1.421296,1.578704,1.351852,1.731481,1.796296,1.750000,1.629630,0.0,4
8,FOODS_1,WI_2,0.884259,0.851852,0.712963,0.949074,0.444444,0.875000,0.851852,0.958333,...,2.657407,2.101852,1.787037,1.824074,1.555556,2.004630,1.976852,1.842593,0.0,4
9,FOODS_1,WI_3,0.703704,0.638889,0.587963,0.453704,0.402778,0.787037,0.629630,0.754630,...,1.787037,1.254630,1.291667,1.222222,1.564815,1.319444,1.458333,1.287037,0.0,0
10,FOODS_2,CA_1,1.693467,1.645729,0.994975,1.195980,0.889447,1.221106,1.391960,1.876884,...,1.962312,1.394472,1.195980,1.072864,1.032663,1.090452,1.575377,1.806533,0.0,0


In [113]:
b = pd.concat([mean_by_dept_store, div_clu0['dbscan2']], axis = 1)
b.loc[b['dbscan_dept_store'] == 0, 'dbscan_dept_store'] = b['dbscan2']
mean_by_dept_store = b.drop(['dbscan2'], axis=1)
mean_by_dept_store
# cluster = [0, 1, 2, 3]
# cluster 0 = 51
# cluster 1 = 4
# cluster 2 = 7
# cluster 3 = 3
# cluster 4 = 5

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,4.0
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778,3.0
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889,3.0
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,0.0
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981,0.0
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184,0.0
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670,0.0
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583,0.0


In [114]:
div_clu0 = mean_by_dept_store[mean_by_dept_store['dbscan_dept_store'] == 0]
div_clu0 = div_clu0.drop(['dept_id', 'store_id', 'dbscan_dept_store'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
3,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,0.402778,0.305556,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,0.550926,0.439815,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
6,0.638889,0.569444,0.523148,0.388889,0.282407,0.495370,0.444444,0.680556,0.615741,0.449074,...,2.009259,1.509259,1.583333,1.500000,1.379630,1.361111,1.643519,1.851852,1.444444,1.361111
9,0.703704,0.638889,0.587963,0.453704,0.402778,0.787037,0.629630,0.754630,0.717593,0.444444,...,1.555556,1.958333,1.787037,1.254630,1.291667,1.222222,1.564815,1.319444,1.458333,1.287037
10,1.693467,1.645729,0.994975,1.195980,0.889447,1.221106,1.391960,1.876884,1.459799,1.404523,...,1.369347,1.535176,1.962312,1.394472,1.195980,1.072864,1.032663,1.090452,1.575377,1.806533
11,0.532663,0.570352,0.346734,0.437186,0.256281,0.258794,0.351759,0.474874,0.437186,0.399497,...,1.281407,1.560302,2.213568,1.316583,1.155779,1.005025,1.258794,1.195980,1.886935,2.097990
13,0.881910,0.871859,0.595477,0.635678,0.603015,0.562814,0.590452,0.776382,0.733668,0.695980,...,0.841709,1.027638,1.379397,1.082915,0.959799,0.766332,0.826633,0.942211,1.015075,1.228643
14,0.979899,0.927136,0.640704,1.050251,0.894472,1.414573,0.728643,1.025126,0.972362,0.806533,...,1.000000,1.052764,1.351759,0.924623,0.821608,0.685930,0.628141,0.786432,0.994975,0.861809
15,1.341709,1.467337,0.979899,1.185930,1.052764,1.728643,0.927136,1.459799,1.449749,1.037688,...,1.060302,1.000000,1.500000,1.349246,1.103015,0.761307,0.994975,0.894472,1.233668,1.298995
16,1.155779,1.188442,0.912060,1.070352,0.924623,1.444724,0.648241,0.829146,0.793970,0.947236,...,1.256281,1.311558,1.957286,1.547739,1.226131,1.113065,0.949749,1.150754,1.283920,1.427136


In [115]:
model = DBSCAN(eps=10,min_samples=3)
model.fit(div_clu0)
clusters = model.fit_predict(div_clu0)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 16), (0, 31), (1, 4)]


In [116]:
dbscan_2 = pd.DataFrame({'dbscan2': clusters})
div_clu0 = mean_by_dept_store[mean_by_dept_store['dbscan_dept_store'] == 0]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, dbscan_2], axis=1)
div_clu0

,index,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store,dbscan2
0,3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,...,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,0.0,-1
1,4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,...,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,0.0,-1
2,6,FOODS_1,TX_3,0.638889,0.569444,0.523148,0.388889,0.282407,0.495370,0.444444,...,1.583333,1.500000,1.379630,1.361111,1.643519,1.851852,1.444444,1.361111,0.0,-1
3,9,FOODS_1,WI_3,0.703704,0.638889,0.587963,0.453704,0.402778,0.787037,0.629630,...,1.787037,1.254630,1.291667,1.222222,1.564815,1.319444,1.458333,1.287037,0.0,-1
4,10,FOODS_2,CA_1,1.693467,1.645729,0.994975,1.195980,0.889447,1.221106,1.391960,...,1.962312,1.394472,1.195980,1.072864,1.032663,1.090452,1.575377,1.806533,0.0,-1
5,11,FOODS_2,CA_2,0.532663,0.570352,0.346734,0.437186,0.256281,0.258794,0.351759,...,2.213568,1.316583,1.155779,1.005025,1.258794,1.195980,1.886935,2.097990,0.0,-1
6,13,FOODS_2,CA_4,0.881910,0.871859,0.595477,0.635678,0.603015,0.562814,0.590452,...,1.379397,1.082915,0.959799,0.766332,0.826633,0.942211,1.015075,1.228643,0.0,0
7,14,FOODS_2,TX_1,0.979899,0.927136,0.640704,1.050251,0.894472,1.414573,0.728643,...,1.351759,0.924623,0.821608,0.685930,0.628141,0.786432,0.994975,0.861809,0.0,0
8,15,FOODS_2,TX_2,1.341709,1.467337,0.979899,1.185930,1.052764,1.728643,0.927136,...,1.500000,1.349246,1.103015,0.761307,0.994975,0.894472,1.233668,1.298995,0.0,-1
9,16,FOODS_2,TX_3,1.155779,1.188442,0.912060,1.070352,0.924623,1.444724,0.648241,...,1.957286,1.547739,1.226131,1.113065,0.949749,1.150754,1.283920,1.427136,0.0,-1


In [ ]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['dbscan2'] == -1, 'dbscan2'] = 5
div_clu0.loc[div_clu0['dbscan2'] == 1, 'dbscan2'] = 6
div_clu0 = div_clu0.set_index('index')
div_clu0

In [118]:
b = pd.concat([mean_by_dept_store, div_clu0['dbscan2']], axis = 1)
b.loc[b['dbscan_dept_store'] == 0, 'dbscan_dept_store'] = b['dbscan2']
mean_by_dept_store = b.drop(['dbscan2'], axis=1)
mean_by_dept_store
# cluster = [0, 1, 2, 3]
# cluster 0 = 31
# cluster 1 = 4
# cluster 2 = 7
# cluster 3 = 3
# cluster 4 = 5
# cluster 5 = 16
# cluster 6 = 4

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,4.0
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778,3.0
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889,3.0
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,5.0
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981,0.0
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184,0.0
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670,0.0
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583,0.0


In [120]:
div_clu0 = mean_by_dept_store[mean_by_dept_store['dbscan_dept_store'] == 0]
div_clu0 = div_clu0.drop(['dept_id', 'store_id', 'dbscan_dept_store'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
13,0.881910,0.871859,0.595477,0.635678,0.603015,0.562814,0.590452,0.776382,0.733668,0.695980,...,0.841709,1.027638,1.379397,1.082915,0.959799,0.766332,0.826633,0.942211,1.015075,1.228643
14,0.979899,0.927136,0.640704,1.050251,0.894472,1.414573,0.728643,1.025126,0.972362,0.806533,...,1.000000,1.052764,1.351759,0.924623,0.821608,0.685930,0.628141,0.786432,0.994975,0.861809
31,1.254808,0.915865,0.846154,0.826923,0.680288,0.918269,0.735577,0.927885,0.997596,0.954327,...,0.786058,1.451923,1.274038,0.699519,0.841346,0.723558,0.757212,0.949519,1.661058,1.396635
33,0.379808,0.555288,0.593750,0.552885,0.598558,0.504808,0.375000,0.663462,0.531250,0.574519,...,0.822115,1.098558,0.944712,0.728365,0.786058,0.728365,0.843750,0.973558,1.242788,1.019231
34,0.579327,0.591346,0.230769,0.550481,0.218750,0.689904,0.194712,0.610577,0.557692,0.298077,...,0.579327,0.497596,0.776442,0.894231,0.504808,0.480769,0.449519,0.562500,0.831731,0.903846
35,0.875000,0.769231,0.466346,0.798077,0.555288,0.718750,0.346154,0.600962,0.620192,0.615385,...,0.793269,0.800481,1.170673,0.692308,0.954327,0.771635,0.680288,0.963942,1.204327,1.237981
36,0.557692,0.502404,0.348558,0.468750,0.302885,0.509615,0.312500,0.548077,0.564904,0.278846,...,0.723558,0.689904,1.064904,0.841346,0.884615,0.771635,0.564904,0.891827,1.069712,1.132212
37,1.437500,1.040865,0.762019,0.314904,0.000000,1.105769,1.745192,1.509615,0.764423,0.567308,...,1.098558,1.276442,1.137019,0.656250,0.625000,0.875000,0.740385,0.968750,1.206731,1.346154
38,0.432692,0.288462,0.228365,0.262019,0.137019,0.391827,0.326923,0.365385,0.151442,0.213942,...,0.596154,0.781250,0.598558,0.538462,0.572115,0.812500,0.598558,0.706731,0.711538,0.588942
39,0.615385,0.822115,0.548077,0.439904,0.168269,0.685096,0.858173,0.959135,0.519231,0.533654,...,0.485577,0.829327,0.706731,0.449519,0.418269,0.689904,0.528846,0.581731,0.656250,0.733173


In [121]:
model = DBSCAN(eps=5,min_samples=3)
model.fit(div_clu0)
clusters = model.fit_predict(div_clu0)
print('dbscan_predict: ', sorted(Counter(clusters).items()))

dbscan_predict:  [(-1, 16), (0, 15)]


In [122]:
dbscan_2 = pd.DataFrame({'dbscan2': clusters})
div_clu0 = mean_by_dept_store[mean_by_dept_store['dbscan_dept_store'] == 0]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, dbscan_2], axis=1)
div_clu0

,index,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store,dbscan2
0,13,FOODS_2,CA_4,0.881910,0.871859,0.595477,0.635678,0.603015,0.562814,0.590452,...,1.379397,1.082915,0.959799,0.766332,0.826633,0.942211,1.015075,1.228643,0.0,-1
1,14,FOODS_2,TX_1,0.979899,0.927136,0.640704,1.050251,0.894472,1.414573,0.728643,...,1.351759,0.924623,0.821608,0.685930,0.628141,0.786432,0.994975,0.861809,0.0,-1
2,31,HOBBIES_1,CA_2,1.254808,0.915865,0.846154,0.826923,0.680288,0.918269,0.735577,...,1.274038,0.699519,0.841346,0.723558,0.757212,0.949519,1.661058,1.396635,0.0,-1
3,33,HOBBIES_1,CA_4,0.379808,0.555288,0.593750,0.552885,0.598558,0.504808,0.375000,...,0.944712,0.728365,0.786058,0.728365,0.843750,0.973558,1.242788,1.019231,0.0,-1
4,34,HOBBIES_1,TX_1,0.579327,0.591346,0.230769,0.550481,0.218750,0.689904,0.194712,...,0.776442,0.894231,0.504808,0.480769,0.449519,0.562500,0.831731,0.903846,0.0,-1
5,35,HOBBIES_1,TX_2,0.875000,0.769231,0.466346,0.798077,0.555288,0.718750,0.346154,...,1.170673,0.692308,0.954327,0.771635,0.680288,0.963942,1.204327,1.237981,0.0,-1
6,36,HOBBIES_1,TX_3,0.557692,0.502404,0.348558,0.468750,0.302885,0.509615,0.312500,...,1.064904,0.841346,0.884615,0.771635,0.564904,0.891827,1.069712,1.132212,0.0,-1
7,37,HOBBIES_1,WI_1,1.437500,1.040865,0.762019,0.314904,0.000000,1.105769,1.745192,...,1.137019,0.656250,0.625000,0.875000,0.740385,0.968750,1.206731,1.346154,0.0,-1
8,38,HOBBIES_1,WI_2,0.432692,0.288462,0.228365,0.262019,0.137019,0.391827,0.326923,...,0.598558,0.538462,0.572115,0.812500,0.598558,0.706731,0.711538,0.588942,0.0,-1
9,39,HOBBIES_1,WI_3,0.615385,0.822115,0.548077,0.439904,0.168269,0.685096,0.858173,...,0.706731,0.449519,0.418269,0.689904,0.528846,0.581731,0.656250,0.733173,0.0,-1


In [123]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['dbscan2'] == -1, 'dbscan2'] = 7
div_clu0 = div_clu0.set_index('index')
div_clu0

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store,dbscan2
index,,,,,,,,,,,,,,,,,,,,,
13,FOODS_2,CA_4,0.881910,0.871859,0.595477,0.635678,0.603015,0.562814,0.590452,0.776382,...,1.379397,1.082915,0.959799,0.766332,0.826633,0.942211,1.015075,1.228643,0.0,7
14,FOODS_2,TX_1,0.979899,0.927136,0.640704,1.050251,0.894472,1.414573,0.728643,1.025126,...,1.351759,0.924623,0.821608,0.685930,0.628141,0.786432,0.994975,0.861809,0.0,7
31,HOBBIES_1,CA_2,1.254808,0.915865,0.846154,0.826923,0.680288,0.918269,0.735577,0.927885,...,1.274038,0.699519,0.841346,0.723558,0.757212,0.949519,1.661058,1.396635,0.0,7
33,HOBBIES_1,CA_4,0.379808,0.555288,0.593750,0.552885,0.598558,0.504808,0.375000,0.663462,...,0.944712,0.728365,0.786058,0.728365,0.843750,0.973558,1.242788,1.019231,0.0,7
34,HOBBIES_1,TX_1,0.579327,0.591346,0.230769,0.550481,0.218750,0.689904,0.194712,0.610577,...,0.776442,0.894231,0.504808,0.480769,0.449519,0.562500,0.831731,0.903846,0.0,7
35,HOBBIES_1,TX_2,0.875000,0.769231,0.466346,0.798077,0.555288,0.718750,0.346154,0.600962,...,1.170673,0.692308,0.954327,0.771635,0.680288,0.963942,1.204327,1.237981,0.0,7
36,HOBBIES_1,TX_3,0.557692,0.502404,0.348558,0.468750,0.302885,0.509615,0.312500,0.548077,...,1.064904,0.841346,0.884615,0.771635,0.564904,0.891827,1.069712,1.132212,0.0,7
37,HOBBIES_1,WI_1,1.437500,1.040865,0.762019,0.314904,0.000000,1.105769,1.745192,1.509615,...,1.137019,0.656250,0.625000,0.875000,0.740385,0.968750,1.206731,1.346154,0.0,7
38,HOBBIES_1,WI_2,0.432692,0.288462,0.228365,0.262019,0.137019,0.391827,0.326923,0.365385,...,0.598558,0.538462,0.572115,0.812500,0.598558,0.706731,0.711538,0.588942,0.0,7


In [124]:
b = pd.concat([mean_by_dept_store, div_clu0['dbscan2']], axis = 1)
b.loc[b['dbscan_dept_store'] == 0, 'dbscan_dept_store'] = b['dbscan2']
mean_by_dept_store = b.drop(['dbscan2'], axis=1)
mean_by_dept_store
# cluster = [0, 1, 2, 3]
# cluster 0 = 15
# cluster 1 = 4
# cluster 2 = 7
# cluster 3 = 3
# cluster 4 = 5
# cluster 5 = 16
# cluster 6 = 4
# cluster 7 = 16

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296,4.0
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778,3.0
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889,3.0
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111,5.0
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981,0.0
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184,0.0
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670,0.0
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583,0.0


In [125]:
dcol=[]
for i in list(mean_by_dept_store):
    if 'd_' in i:
        dcol.append(i)
        
mean_by_dept_store = mean_by_dept_store.drop(dcol, axis=1)
mean_by_dept_store = mean_by_dept_store.astype({'dbscan_dept_store':'int'})
mean_by_dept_store

,dept_id,store_id,dbscan_dept_store
0,FOODS_1,CA_1,4
1,FOODS_1,CA_2,3
2,FOODS_1,CA_3,3
3,FOODS_1,CA_4,5
4,FOODS_1,TX_1,5
...,...,...,...
65,HOUSEHOLD_2,TX_2,0
66,HOUSEHOLD_2,TX_3,0
67,HOUSEHOLD_2,WI_1,0
68,HOUSEHOLD_2,WI_2,0


In [126]:
# cluster 8개
train_clu = pd.merge(train_df, mean_by_dept_store, on=['dept_id', 'store_id'])
train_clu

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,dbscan_dept_store
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,0,0,0,0,3,3,0,1,5
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,0,0,0,0,0,5
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,0,0,0,2,3,0,1,5
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,4,0,1,3,0,2,6,5
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,2,1,0,0,2,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,3,0,1,1,0,0,1,1,2
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,1,0,1,0,2
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,1,2,0,1,0,1,0,2,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,4,6,0,1,1,1,0,2


In [127]:
train_clu.to_csv(raw_data_dir+'sales_train_deptstore_dbscan_evaluation.csv')